In [3]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


we need to import the request and access wikipedia

In [1]:
pip install geopy

     |████████████████████████████████| 102kB 13.7MB/s ta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib
Note: you may need to restart the kernel to use updated packages.


In [50]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import numpy as np # library to handle data in a vectorized manner

print('Libraries imported.')


Libraries imported.


import BS for table

In [3]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
from bs4 import BeautifulSoup
bsoup = BeautifulSoup(website_url,'html')
#print(soup.prettify())

In [5]:
My_table = bsoup.find('table',{'class':'wikitable sortable'})
#My_table

here, we need to look at the table, find the body of the table, find the elements between tr and td, create a DF

we reset the index

In [6]:
table_body = My_table.find('tbody')
rows = table_body.find_all('tr')
l=[]
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    l.append(cols)
df_PC=pd.DataFrame(l, columns=["Postal Code","Borough","Neighborhood"])
df_PC=df_PC.drop(df_PC.index[[0]]).reset_index(drop=True)
#df_PC

now, we need to get rid of not assigned in borough

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [7]:
df_drop = df_PC.drop(df_PC[df_PC.Borough == 'Not assigned'].index).reset_index(drop=True)
#df_drop

now, we need to replace the neighborood with not assigned

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [8]:
for index, row in df_drop.iterrows():
    if row ['Neighborhood']=='Not assigned':
        row ['Neighborhood']= row ['Borough']
#df_drop


next, we need to join the neighborood with equal postal code; we group the dt into postal code and borough and we map the values using lambda function

In [9]:
df_final = df_drop.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(lambda x: ', '.join(x.astype(str))).reset_index()
df_final.head(5)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [10]:
df_final.shape

(103, 3)

In [11]:
DF_LL = pd.read_csv("https://cocl.us/Geospatial_data")
DF_LL.columns = ['Postal Code', 'Latitude', 'Longitude']
DF_LL.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


now we can merge the two data frame on the column postalcodes

In [13]:
DF_merged=pd.merge(DF_LL,df_final , on='Postal Code')
DF_merged.head(5)

,Postal Code,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


this new DF must be reordered

In [19]:
DF_merged=DF_merged[['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
DF_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


we need to create a map to see Toronto areas

In [17]:
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(DF_merged['Latitude'], DF_merged['Longitude'], DF_merged['Borough'], DF_merged['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

we now need to connect with Foursquare Credentials and Version

In [18]:
CLIENT_ID = '22LA0W1UYAA2MYXY0WMLO3ZR1Y0Y2D5TGXELMPXQEOC1OF4S' # your Foursquare ID
CLIENT_SECRET = '1ZLFGQJ0EJHIGNRTYC4TVVSB3LT0ACLLAQJ32KTN5LUQ1MAP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 22LA0W1UYAA2MYXY0WMLO3ZR1Y0Y2D5TGXELMPXQEOC1OF4S
CLIENT_SECRET:1ZLFGQJ0EJHIGNRTYC4TVVSB3LT0ACLLAQJ32KTN5LUQ1MAP


start with first neighborhood

In [20]:
DF_merged.loc[0, 'Neighborhood']

'Rouge, Malvern'

now we look at their position

In [22]:
neighborhood_latitude = DF_merged.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = DF_merged.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = DF_merged.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


now we are interested in looking at first 200 venues close to these locations in a radius of 1 km

In [29]:
LIMIT = 200 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=22LA0W1UYAA2MYXY0WMLO3ZR1Y0Y2D5TGXELMPXQEOC1OF4S&client_secret=1ZLFGQJ0EJHIGNRTYC4TVVSB3LT0ACLLAQJ32KTN5LUQ1MAP&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=1000&limit=200'

In [34]:
results = requests.get(url).json()


In [31]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777
2,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607
3,Wendy's,Fast Food Restaurant,43.802008,-79.198080
4,Harvey's,Fast Food Restaurant,43.800106,-79.198258
5,Wendy's,Fast Food Restaurant,43.807448,-79.199056
6,Tim Hortons,Coffee Shop,43.802000,-79.198169
7,Lee Valley,Hobby Shop,43.803161,-79.199681
8,Bus Stop: 85 & 116,Bus Station,43.802198,-79.199389
9,MMA World Academy,Martial Arts Dojo,43.800259,-79.195227


In [33]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

19 venues were returned by Foursquare.


now we can look for all the venues in toronto within 1 km

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
toronto_venues=getNearbyVenues(names=DF_merged['Neighborhood'],
                                   latitudes=DF_merged['Latitude'],
                                   longitudes=DF_merged['Longitude']
                                  )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [42]:
print(toronto_venues.shape)
toronto_venues.head()

(4869, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,"Rouge, Malvern",43.806686,-79.194353,Caribbean Wave,43.798558,-79.195777,Caribbean Restaurant
2,"Rouge, Malvern",43.806686,-79.194353,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store
3,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
4,"Rouge, Malvern",43.806686,-79.194353,Harvey's,43.800106,-79.198258,Fast Food Restaurant


for each neighborhood we can count how many venues

In [44]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,46,46,46,46,46,46
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",28,28,28,28,28,28
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",16,16,16,16,16,16
"Alderwood, Long Branch",29,29,29,29,29,29
"Bathurst Manor, Downsview North, Wilson Heights",28,28,28,28,28,28
Bayview Village,13,13,13,13,13,13
"Bedford Park, Lawrence Manor East",39,39,39,39,39,39
Berczy Park,100,100,100,100,100,100


we can observe how downtown neighborhoods (queens park, little portugal, etc..) present the highest number of venues

wer can now look at each neighborhood and see the frequency of the first 5 venue in each neighborhood in Toronto

In [47]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.shape
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped
toronto_grouped.shape
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    
    
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

----Adelaide, King, Richmond----
              venue  freq
0       Coffee Shop  0.07
1              Café  0.06
2             Hotel  0.05
3           Theater  0.04
4  Sushi Restaurant  0.03


----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.13
1         Shopping Mall  0.09
2  Caribbean Restaurant  0.04
3            Restaurant  0.04
4           Supermarket  0.04


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0  Chinese Restaurant  0.21
1              Bakery  0.07
2                Park  0.07
3            Pharmacy  0.07
4         Pizza Place  0.07


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
           venue  freq
0    Pizza Place  0.25
1  Grocery Store  0.19
2     Beer Store  0.06
3           Park  0.06
4       Bus Line  0.06


----Alderwood, Long Branch----
               venue  freq
0           Pharmacy  0.10
1     Discount Store  0.10
2  C

for the first 15 most common venues

In [106]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Theater,Sushi Restaurant,Japanese Restaurant,Cosmetics Shop,Pizza Place,Deli / Bodega,Concert Hall,Clothing Store,Bakery,Asian Restaurant,Bar,Steakhouse
1,Agincourt,Chinese Restaurant,Shopping Mall,Supermarket,Bakery,Restaurant,Caribbean Restaurant,Seafood Restaurant,Sri Lankan Restaurant,Filipino Restaurant,Breakfast Spot,Lounge,Malay Restaurant,Mediterranean Restaurant,Skating Rink,Shanghai Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Park,Pizza Place,Pharmacy,Bakery,Event Space,Malay Restaurant,Shopping Mall,Shop & Service,Noodle House,Caribbean Restaurant,Korean Restaurant,BBQ Joint,Dessert Shop,Hobby Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Grocery Store,Pharmacy,Fast Food Restaurant,Bus Line,Hardware Store,Fried Chicken Joint,Park,Coffee Shop,Sandwich Place,Beer Store,Electronics Store,Drugstore,Dry Cleaner,Dumpling Restaurant
4,"Alderwood, Long Branch",Pharmacy,Discount Store,Pizza Place,Convenience Store,Grocery Store,Park,Pool,Athletics & Sports,Moroccan Restaurant,Skating Rink,Donut Shop,Shopping Mall,Liquor Store,Sandwich Place,Gas Station


now we run the  k-means clustering

In [108]:
km_clusters = 30

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=km_clusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50] 

array([ 3,  0,  0, 10, 11, 29, 12,  6,  3, 12, 11,  3, 29,  6, 12, 29,  1,
       29,  6,  3,  3, 26, 29, 11, 11, 13, 14,  3,  6,  6, 29, 27,  3, 29,
        6,  1,  5, 11, 17, 23, 11,  1,  8, 29,  3, 29, 12,  6, 10,  3],
      dtype=int32)

we can name the clusters and merge all teh dataframes got so far with lat, lon, venues, names


In [109]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged =  DF_merged

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1.0,Fast Food Restaurant,Arts & Crafts Store,Spa,Bus Station,Martial Arts Dojo,Fruit & Vegetable Store,Caribbean Restaurant,Bakery,Chinese Restaurant,Paper / Office Supplies Store,Park,Trail,Greek Restaurant,Hobby Shop,Sandwich Place
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,7.0,Breakfast Spot,Park,Burger Joint,Playground,Italian Restaurant,Ethiopian Restaurant,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Doner Restaurant,Falafel Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,10.0,Pizza Place,Fast Food Restaurant,Coffee Shop,Burger Joint,Greek Restaurant,Bank,Discount Store,Sandwich Place,Supermarket,Beer Store,Electronics Store,Food & Drink Shop,Liquor Store,Sports Bar,Grocery Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,16.0,Coffee Shop,Electronics Store,Park,Fast Food Restaurant,Chinese Restaurant,Indian Restaurant,Pharmacy,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio,Empanada Restaurant,Dog Run
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,6.0,Bakery,Coffee Shop,Pharmacy,Indian Restaurant,Yoga Studio,Athletics & Sports,Sporting Goods Shop,Music Store,Fried Chicken Joint,Caribbean Restaurant,Chinese Restaurant,Bank,Fast Food Restaurant,Thai Restaurant,Burger Joint


create a map of venues

In [110]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(km_clusters)
ys = [i + x + (i*x)**2 for i in range(km_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
      #  color=rainbow[cluster-1],
        fill=True,
       # fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [111]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
12,Scarborough,0.0,Chinese Restaurant,Shopping Mall,Supermarket,Bakery,Restaurant,Caribbean Restaurant,Seafood Restaurant,Sri Lankan Restaurant,Filipino Restaurant,Breakfast Spot,Lounge,Malay Restaurant,Mediterranean Restaurant,Skating Rink,Shanghai Restaurant
14,Scarborough,0.0,Chinese Restaurant,Park,Pizza Place,Pharmacy,Bakery,Event Space,Malay Restaurant,Shopping Mall,Shop & Service,Noodle House,Caribbean Restaurant,Korean Restaurant,BBQ Joint,Dessert Shop,Hobby Shop
15,Scarborough,0.0,Chinese Restaurant,Coffee Shop,Bakery,Fast Food Restaurant,Tennis Court,Sandwich Place,Bank,Other Great Outdoors,Nail Salon,Shopping Mall,Camera Store,Auto Garage,Intersection,Hotpot Restaurant,Pharmacy


In [112]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Scarborough,1.0,Fast Food Restaurant,Arts & Crafts Store,Spa,Bus Station,Martial Arts Dojo,Fruit & Vegetable Store,Caribbean Restaurant,Bakery,Chinese Restaurant,Paper / Office Supplies Store,Park,Trail,Greek Restaurant,Hobby Shop,Sandwich Place
35,East York,1.0,Fast Food Restaurant,Pizza Place,Brewery,Gym / Fitness Center,Gastropub,Bank,Bakery,Café,Athletics & Sports,Rock Climbing Spot,Coffee Shop,Intersection,Pet Store,Pharmacy,Soccer Stadium
40,East York,1.0,Coffee Shop,Café,Greek Restaurant,Pizza Place,Pharmacy,Ethiopian Restaurant,Bakery,Convenience Store,Burger Joint,Pub,Breakfast Spot,Spa,Gastropub,Beer Bar,Sandwich Place
41,East Toronto,1.0,Greek Restaurant,Coffee Shop,Pub,Café,Fast Food Restaurant,Pizza Place,Ice Cream Shop,Italian Restaurant,Bank,Bakery,Ramen Restaurant,Spa,Sandwich Place,Diner,Yoga Studio
74,York,1.0,Mexican Restaurant,Pharmacy,Park,Japanese Restaurant,Pizza Place,Coffee Shop,Discount Store,Café,Falafel Restaurant,Bus Stop,Fast Food Restaurant,Food Truck,Market,Bank,Bakery
76,West Toronto,1.0,Coffee Shop,Café,Park,Bar,Sushi Restaurant,Convenience Store,Supermarket,Pharmacy,Gourmet Shop,Bakery,Italian Restaurant,Brewery,Portuguese Restaurant,Music Venue,Gym
82,West Toronto,1.0,Bar,Café,Coffee Shop,Park,Convenience Store,Italian Restaurant,Sushi Restaurant,Bakery,Pizza Place,Thai Restaurant,Metro Station,Breakfast Spot,Nail Salon,Mexican Restaurant,Arts & Crafts Store
84,West Toronto,1.0,Coffee Shop,Café,Pizza Place,Bakery,Italian Restaurant,Falafel Restaurant,Park,Spa,Latin American Restaurant,Pub,Gastropub,Sushi Restaurant,Pilates Studio,Pharmacy,Dessert Shop
88,Etobicoke,1.0,Park,Mexican Restaurant,Indian Restaurant,Falafel Restaurant,Café,Skating Rink,Liquor Store,Fried Chicken Joint,Bakery,Fast Food Restaurant,Dessert Shop,Italian Restaurant,Sandwich Place,Restaurant,American Restaurant


In [113]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
43,East Toronto,3.0,Coffee Shop,Bar,Bakery,American Restaurant,Café,Vietnamese Restaurant,Brewery,Italian Restaurant,Diner,Sushi Restaurant,French Restaurant,Thai Restaurant,Park,Gastropub,Arts & Crafts Store
54,Downtown Toronto,3.0,Coffee Shop,Cosmetics Shop,Clothing Store,Italian Restaurant,Café,Restaurant,Middle Eastern Restaurant,Tea Room,Gastropub,Bubble Tea Shop,Ramen Restaurant,Hotel,Diner,Japanese Restaurant,Plaza
55,Downtown Toronto,3.0,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Breakfast Spot,Seafood Restaurant,Gastropub,Cosmetics Shop,Farmers Market,Cheese Shop,Beer Bar,Cocktail Bar,Park,Bakery
56,Downtown Toronto,3.0,Coffee Shop,Hotel,Café,Beer Bar,Japanese Restaurant,Park,Restaurant,Cocktail Bar,Italian Restaurant,Steakhouse,Breakfast Spot,Bakery,Farmers Market,Seafood Restaurant,Cheese Shop
57,Downtown Toronto,3.0,Coffee Shop,Café,Ice Cream Shop,Bubble Tea Shop,Italian Restaurant,Park,Vegetarian / Vegan Restaurant,Ramen Restaurant,Chinese Restaurant,Tea Room,Breakfast Spot,Theater,Cosmetics Shop,Spa,Arts & Crafts Store
58,Downtown Toronto,3.0,Coffee Shop,Café,Hotel,Theater,Sushi Restaurant,Japanese Restaurant,Cosmetics Shop,Pizza Place,Deli / Bodega,Concert Hall,Clothing Store,Bakery,Asian Restaurant,Bar,Steakhouse
59,Downtown Toronto,3.0,Coffee Shop,Hotel,Café,Aquarium,Italian Restaurant,Scenic Lookout,Bakery,Park,Brewery,Fried Chicken Joint,Japanese Restaurant,Gym,Steakhouse,Baseball Stadium,Theater
60,Downtown Toronto,3.0,Coffee Shop,Hotel,Café,Italian Restaurant,Steakhouse,American Restaurant,Restaurant,Concert Hall,Plaza,Bakery,Beer Bar,Deli / Bodega,Park,Theater,Monument / Landmark
61,Downtown Toronto,3.0,Coffee Shop,Café,Hotel,American Restaurant,Concert Hall,Beer Bar,Gastropub,Restaurant,Steakhouse,Japanese Restaurant,Deli / Bodega,Bakery,Thai Restaurant,Plaza,Cocktail Bar
65,Central Toronto,3.0,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Restaurant,Bakery,Italian Restaurant,Pizza Place,Pub,Museum,Grocery Store,Gym,Mexican Restaurant,Deli / Bodega,Indian Restaurant,History Museum


In [114]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
23,North York,4.0,Park,Restaurant,Diner,Coffee Shop,Bank,Chinese Restaurant,French Restaurant,Business Service,Pet Store,Optical Shop,Grocery Store,Playground,Convenience Store,Dog Run,Electronics Store
50,Downtown Toronto,4.0,Coffee Shop,Park,Bank,Grocery Store,Smoothie Shop,Building,Metro Station,Playground,Mattress Store,Breakfast Spot,Bistro,Candy Store,Office,Sandwich Place,BBQ Joint


using 5 clusters it seems we cannot have homogeneous data division, we can increase the clusters

In [118]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 9, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
20,North York,9.0,Park,Pool,Yoga Studio,Ethiopian Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Dog Run,Falafel Restaurant,Farm


we need to increase the number of clusters to divide Toronto's neighborhoods into areas close to each other